# Import and setup

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import few

from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode import SchwarzEccFlux, KerrEccEqFlux
# from few.amplitude.romannet import RomanAmplitude
from few.amplitude.ampinterp2d import AmpInterpKerrEqEcc
from few.summation.interpolatedmodesum import InterpolatedModeSum


from few.utils.ylm import GetYlms
from few.utils.modeselector import ModeSelector
from few.summation.interpolatedmodesum import CubicSplineInterpolant
from few import get_file_manager

from few.waveform import (
    FastKerrEccentricEquatorialFlux,
    FastSchwarzschildEccentricFlux, 
    SlowSchwarzschildEccentricFlux, 
    Pn5AAKWaveform,
    GenerateEMRIWaveform
)

from few.utils.utility import get_fundamental_frequencies

import GWfuncs
import gc
import pickle
import os
# import cupy as cp
import multiprocessing as mp
from multiprocessing import Queue, Process
from functools import partial
from SNR_tutorial_utils import LISA_Noise
from lisatools.sensitivity import *


# import pandas as pd
# tune few configuration
cfg_set = few.get_config_setter(reset=True)
cfg_set.set_log_level("info");

In [3]:
# Parameters
M = 1e6
mu = 1e1 
a = 0.5
p0 = 9.5
e0 = 0.2
theta = np.pi / 3.0 
phi = np.pi / 4.0  
dt = 10.0
T = 1
xI0 = 1.0 
#in the paper xI0 = 0.866, but that would be non-equatorial case

traj = EMRIInspiral(func=KerrEccEqFlux, force_backend="cuda12x")
amp = AmpInterpKerrEqEcc(lmax=10,nmax=55) # default lmax=10, nmax=55
interpolate_mode_sum = InterpolatedModeSum()
ylm_gen = GetYlms(assume_positive_m=False)

use_gpu = False
func = "KerrEccentricEquatorial"

# keyword arguments for inspiral generator 
inspiral_kwargs={
        "err": 1e-10,
        "func": func,
        "DENSE_STEPPING": 0,  # we want a sparsely sampled trajectory
        "include_minus_m": False, 
        "use_gpu" : use_gpu,
        "force_backend": "cuda12x"  # Force GPU
        # "buffer_length": int(1e4),  # all of the trajectories will be well under len = 1000
        # diff in v2.0: max init length => buffer length
    }

# keyword arguments for inspiral generator 
amplitude_kwargs = {
    "force_backend": "cuda12x"  # Force GPU
    # "buffer_length": int(1e3),  # all of the trajectories will be well under len = 1000
    # "use_gpu": use_gpu  # GPU is available in this class
}

# keyword arguments for Ylm generator (GetYlms)
Ylm_kwargs = {
    "force_backend": "cuda12x",  # Force GPU
    "assume_positive_m": True  # if we assume positive m, it will generate negative m for all m>0
}

# keyword arguments for summation generator (InterpolatedModeSum)
sum_kwargs = {
    "force_backend": "cuda12x",  # Force GPU
    # "use_gpu": use_gpu,  # GPU is available for this type of summation
    "pad_output": False,
}

# Kerr eccentric flux
few = FastKerrEccentricEquatorialFlux(
    inspiral_kwargs=inspiral_kwargs,
    amplitude_kwargs=amplitude_kwargs,
    Ylm_kwargs=Ylm_kwargs,
    sum_kwargs=sum_kwargs,
    use_gpu=use_gpu,
)


# Generate waveform

In [5]:
# Calc trajectory
t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(M, mu, a, p0, e0, 1.0)
# the arguments: M, mu, a, p0, e0, x0

# t_gpu = cp.asarray(t)

# Get amplitudes along trajectory
teuk_modes = amp(a, p, e, x)

# Get Ylms
ylms = ylm_gen(amp.unique_l, amp.unique_m, theta, phi).copy()[amp.inverse_lm]

# need to prepare arrays for sum with all modes due to +/- m setup
ls = amp.l_arr[: teuk_modes.shape[1]]
ms = amp.m_arr[: teuk_modes.shape[1]]
ns = amp.n_arr[: teuk_modes.shape[1]]

keep_modes = np.arange(teuk_modes.shape[1])
temp2 = keep_modes * (keep_modes < amp.num_m0) + (keep_modes + amp.num_m_1_up) * (
    keep_modes >= amp.num_m0
)

ylmkeep = np.concatenate([keep_modes, temp2])
ylms_in = ylms[ylmkeep]
teuk_modes_in = teuk_modes

# summation
waveform1 = interpolate_mode_sum(
    t,
    teuk_modes_in,
    ylms_in,
    traj.integrator_spline_t,
    traj.integrator_spline_phase_coeff[:, [0, 2]],
    amp.m_arr,
    amp.n_arr,    
    dt=dt,
    T=T,
)

N = int(len(waveform1)) 
gwf = GWfuncs.GravWaveAnalysis(N=N,dt=dt)
# Calculate distance dimensionless
dist = 1.0 #Gpc
factor = gwf.dist_factor(dist, mu)
waveform1_scaled = waveform1/factor

In [6]:
hfull_f = gwf.freq_wave(waveform1_scaled)

SNR_ref = gwf.SNR(hfull_f)
print("SNR:", SNR_ref)
print("SNR squared:", SNR_ref**2)

SNR: 38.19806409907605
SNR squared: 1459.0921009171225


In [7]:
# Convert T to standard units
YRSID_SI = 31558149.763545603  # 1 sidereal year in seconds
T_sd = 1.0 * YRSID_SI  # 1 sidereal year in seconds (~31,558,150 seconds)
print("Observation time in seconds:", T_sd)

Observation time in seconds: 31558149.763545603


In [8]:
N_traj = teuk_modes.shape[0]  # number of trajectory points
print("Number of trajectory points:", N_traj)
delta_T = T_sd / N_traj  # time step in seconds
print("Time step in seconds", delta_T)

Number of trajectory points: 14
Time step in seconds 2254153.5545389717


In [9]:
# Get mode labels
mode_labels = [f"({l},{m},{n})" for l,m,n in zip(amp.l_arr, amp.m_arr, amp.n_arr)]

# Generate mode frequencies 

Using *get_fundamental_frequencies* instead

In [ ]:
OmegaPhi, OmegaTheta, OmegaR = get_fundamental_frequencies(a, p, e, x)

In [11]:
OmegaPhi

array([0.03205396, 0.03205396, 0.03205397, 0.03205403, 0.03205441,
       0.03205666, 0.03207016, 0.03215159, 0.03265547, 0.03393851,
       0.03541557, 0.03776285, 0.0400312 , 0.0400364 ])

In [16]:
gw_frequencies_per_mode = []
mode_frequencies = {}

for idx in range(len(mode_labels)):
    l = amp.l_arr[idx]
    m = amp.m_arr[idx] 
    n = amp.n_arr[idx]
    
    # Calculate GW frequencies
    # k = 0 for equatorial case
    f_gw = m * OmegaPhi + n * OmegaR
    
    gw_frequencies_per_mode.append(f_gw)

In [17]:
gw_phase_per_mode = []
for idx in range(len(mode_labels)):
    l = amp.l_arr[idx]
    m = amp.m_arr[idx] 
    n = amp.n_arr[idx]
    
    # Calculate GW phases per mode
    phi_mode = m * Phi_phi + n * Phi_r
    
    gw_phase_per_mode.append(phi_mode)

# Calculate inner product

In [21]:
idx_i = 1165 # 220
idx_j = 1166 # 221
mode_labels[idx_i], mode_labels[idx_j]

('(2,2,0)', '(2,2,1)')

In [24]:
# Get complex amplitudes for the two modes
A0 = teuk_modes[:, idx_i]
A1 = teuk_modes[:, idx_j]
print("A_0:", A0)
print("A_1:", A1)

A_0: [0.44924664-0.12386828j 0.44924666-0.12386829j 0.44924679-0.12386835j
 0.44924758-0.12386868j 0.44925227-0.1238707j  0.44928045-0.12388277j
 0.44944961-0.1239553j  0.45046868-0.12439248j 0.45673448-0.12709187j
 0.47238826-0.13391732j 0.48990391-0.14168195j 0.51672206-0.15379251j
 0.54157349-0.16520098j 0.5416294 -0.1652268j ]
A_1: [0.26093475-0.08208452j 0.26093475-0.08208452j 0.26093475-0.08208453j
 0.26093478-0.08208459j 0.26093493-0.08208497j 0.26093583-0.08208722j
 0.26094125-0.08210074j 0.26097355-0.08218193j 0.26115894-0.08267163j
 0.26153211-0.08382475j 0.26182414-0.08499903j 0.26209755-0.08657027j
 0.26226481-0.08779671j 0.26226517-0.08779923j]


In [27]:
# Get sensitivity for the two modes
Sn0 = get_sensitivity(gw_frequencies_per_mode[idx_i], sens_fn=LISASens, return_type="PSD")
Sn1 = get_sensitivity(gw_frequencies_per_mode[idx_j], sens_fn=LISASens, return_type="PSD")

In [28]:
barA0 = A0 / np.sqrt(Sn0)
barA0 

array([1.02549867e+19-2.82755056e+18j, 1.02549867e+19-2.82755064e+18j,
       1.02549868e+19-2.82755112e+18j, 1.02549872e+19-2.82755397e+18j,
       1.02549899e+19-2.82757110e+18j, 1.02550059e+19-2.82767385e+18j,
       1.02551009e+19-2.82829052e+18j, 1.02556444e+19-2.83199505e+18j,
       1.02578971e+19-2.85438341e+18j, 1.02556975e+19-2.90738695e+18j,
       1.02409945e+19-2.96173196e+18j, 1.01964316e+19-3.03477423e+18j,
       1.01348593e+19-3.09152628e+18j, 1.01347015e+19-3.09164215e+18j])

In [29]:
barA1 = A1 / np.sqrt(Sn1)
barA1

array([4.53923442e+18-1.42794654e+18j, 4.53923422e+18-1.42794650e+18j,
       4.53923303e+18-1.42794629e+18j, 4.53922586e+18-1.42794498e+18j,
       4.53918284e+18-1.42793715e+18j, 4.53892470e+18-1.42789019e+18j,
       4.53737508e+18-1.42760812e+18j, 4.52804853e+18-1.42590599e+18j,
       4.47102665e+18-1.41533376e+18j, 4.33116191e+18-1.38819886e+18j,
       4.17947330e+18-1.35683127e+18j, 3.95817442e+18-1.30737668e+18j,
       3.76600900e+18-1.26072270e+18j, 3.76559135e+18-1.26061737e+18j])

In [35]:
phase01 = np.abs(gw_phase_per_mode[idx_i] - gw_phase_per_mode[idx_j]) < 1.0 
phase01

array([ True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False])

In [39]:
crossprod01 = np.conj(barA0[phase01]) * barA1[phase01]
crossprod01

array([5.05873796e+37-1.80865793e+36j, 5.05873777e+37-1.80865778e+36j,
       5.05873658e+37-1.80865688e+36j])

In [40]:
inner_contrib_01 = np.sum(crossprod01) * delta_T * 1/(factor**2)
np.real(inner_contrib_01)

np.float64(78.34048759987516)

In [41]:
selfprod00 = np.conj(barA0)*barA0 
selfprod00

array([1.13159794e+38-1.62405645e+21j, 1.13159795e+38-1.62211251e+21j,
       1.13159799e+38-1.27729898e+21j, 1.13159824e+38-2.21577627e+21j,
       1.13159976e+38+1.91719498e+21j, 1.13160885e+38+1.85324414e+21j,
       1.13166323e+38+1.29929196e+21j, 1.13198438e+38-1.21034929e+21j,
       1.13371958e+38+9.08865403e+19j, 1.13632229e+38+7.35647173e+20j,
       1.13649824e+38-3.90484599e+20j, 1.13177072e+38-1.32351577e+21j,
       1.12272908e+38+1.46226910e+21j, 1.12270425e+38-1.88145804e+21j])

In [43]:
inner_contrib_00 = np.sum(selfprod00) * delta_T * 1/(factor**2)
np.real(inner_contrib_00)

np.float64(817.5147332029419)

In [45]:
np.real(np.sum(barA0**2) * delta_T * 1/(factor**2))

np.float64(696.0118515675955)

In [47]:
np.real(np.sum(np.conj(barA1)*barA1) * delta_T * 1/(factor**2))

np.float64(150.69417288688206)

In [46]:
np.real(np.sum(barA1**2) * delta_T * 1/(factor**2))

np.float64(122.85587955544625)

# Reference values

In [49]:
indices = [1165, 1166]

In [50]:
waveform_per_mode = []
for idx in indices:
    l = amp.l_arr[idx]
    m = amp.m_arr[idx]
    n = amp.n_arr[idx]
    print('Mode: ', mode_labels[idx])

    if m >= 0:
        # For m >= 0, directly use the mode
        teuk_modes_single = teuk_modes[:, [idx]]
        ylms_single = ylms[[idx]]
        m_arr = amp.m_arr[[idx]]
    else:
        # Finding corresponding m>0 mode instead of mapping
        print('NEGATIVE M MODE')
        pos_m_mask = (amp.l_arr == l) & (amp.m_arr == -m) & (amp.n_arr == n) 
        print(amp.l_arr[pos_m_mask], amp.m_arr[pos_m_mask], amp.n_arr[pos_m_mask])
        pos_m_idx = np.where(pos_m_mask)[0]
        print(pos_m_idx)
        
        teuk_modes_single = (-1)**l * np.conj(teuk_modes[:, [pos_m_idx]])
        print(teuk_modes_single)
        # ylms_single = (-1)**(-m) * np.conj(ylms[[pos_m_idx]])
        ylms_single = ylms[[idx]]
        print(ylms_single)
        m_arr = np.abs(amp.m_arr[[idx]])  # To pass positive m 

    waveform = interpolate_mode_sum(
        t,
        teuk_modes_single,
        ylms_single,
        traj.integrator_spline_t,
        traj.integrator_spline_phase_coeff[:, [0, 2]],
        m_arr,  
        amp.n_arr[[idx]], 
        dt=dt,
        T=T
    )
    waveform_per_mode.append(waveform/factor)

Mode:  (2,2,0)
Mode:  (2,2,1)


In [51]:
# Convert each waveform to frequency domain
hf_per_mode = [gwf.freq_wave(waveform) for waveform in waveform_per_mode]

In [54]:
gwf.inner(hf_per_mode[0], hf_per_mode[1])

np.float64(0.006326338103142443)

In [55]:
gwf.inner(hf_per_mode[0], hf_per_mode[0])

np.float64(585.8239700556379)

In [56]:
gwf.inner(hf_per_mode[1], hf_per_mode[1])

np.float64(466.6345702182539)